In [1]:
import sys
sys.path.append("C:/Users/Jia-Wei Chang/hu_file/ResNet18")
#自定義資料取得
import ESC50_puliic_getDataset as epg
#自定義模組
import ESC50_puliic_model as epm

#資料處理套件
import numpy as np
#畫圖套件
import matplotlib.pyplot as plt
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

In [2]:
#測試模型對於電鋸資料集的準確度
def test_model(inputs_shape, test_loader):
    #固定模型不改變權重
    model.eval() 
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.view(inputs_shape)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += 1
            correct += (predicted == labels).sum().item()

        print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
        return 100 * correct / total

#訓練模型
def train_model(model, loss_func, optimizer, input_shape, num_epochs, train_loader, val_loader, test_loader1, test_loader3, test_loader5):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    val_loss_array = []
    val_accuracy_array = []
    test1_accuracy_array = []
    test3_accuracy_array = []
    test5_accuracy_array = []
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #包裝參數 重組形狀
            train = Variable(inputs.view(input_shape))
            labels = Variable(labels)
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(train)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #驗證準確度
        correct_val= 0
        #驗證資料總筆數
        total_val = 0
        
        for data in val_loader:
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #包裝參數 重組形狀
            val = Variable(inputs.view(input_shape))
            #向前傳播
            outputs = model(val)
            #計算loss
            val_loss = loss_func(outputs, labels)
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_val += len(labels)
            #計算判斷正確的筆數
            correct_val += (predicted == labels).float().sum()
        #計算這次驗證的準確度
        val_accuracy = 100 * correct_val / float(total_val)
        #紀錄這次驗證的準確度
        val_accuracy_array.append(val_accuracy)
        #紀錄這次驗證的loss
        val_loss_array.append(val_loss.data)
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy, val_loss.data, val_accuracy))
        #調度器走一步
        scheduler.step()
        #測試電鋸資料集 1S 3S 5S
        test1_accuracy_array.append(test_model((-1,1,128,87), test_loader1))
        test3_accuracy_array.append(test_model((-1,1,128,259), test_loader3))
        test5_accuracy_array.append(test_model((-1,1,128,431), test_loader5))
    return train_loss_array, train_accuracy_array, val_loss_array, val_accuracy_array, test1_accuracy_array, test3_accuracy_array, test5_accuracy_array

In [3]:
train_loss_array = []
train_accuracy_array = []
val_loss_array = []
val_accuracy_array = []
test1_accuracy_array = []
test3_accuracy_array = []
test5_accuracy_array = []
#----------------------------------
#訓練相關參數
#指定訓練位置
device = torch.device('cuda:0')
#訓練次數
num_epochs = 30
#聲音長度
sound = [1, 3, 5]
#批次大小
BATCH_SIZE = 64
#分類數量
num_class = 10
#學習率
LR = 0.0002
#輸入形狀
input_shape = [(-1,1,128,87), (-1,1,128,259), (-1,1,128,431)]

#----------------------------------
#設定存檔位置
f = open("train.txt", "w")
#k-fold訓練
for fold in range(1,6):
    train_loss_TL = []
    train_accuracy_TL = []
    val_loss_TL = []
    val_accuracy_TL = []
    test1_accuracy_TL = []
    test3_accuracy_TL = []
    test5_accuracy_TL = []
    #建立模型
    model = epm.ResNet_18(nums_class = num_class)
    #指定訓練位置
    model.to(device)
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    
    #資料抓入
    test_loader1 = epg.getTestDataset(sound[0])
    test_loader3 = epg.getTestDataset(sound[1])
    test_loader5 = epg.getTestDataset(sound[2])
    
    for TL in range(2,-1,-1):
        print("----------------------------------------------------------------------------------------------------------------")
        print("TL: ", TL+1, "fold: ", fold)
        print("----------------------------------------------------------------------------------------------------------------")
        #資料抓入
        train, val = epg.getTrainDataset(fold, sound[TL], num_class)
        #分批方法
        train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = True)
        val_loader = torch.utils.data.DataLoader(val, batch_size = BATCH_SIZE, shuffle = True)
        
        #優化器
        optimizer = torch.optim.Adam(model.parameters(), lr=LR)
        #調度器
        scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
        #訓練模型
        train_loss, train_accuracy, val_loss, val_accuracy, test1_accuracy, test3_accuracy, test5_accuracy = train_model(model, loss_func, optimizer, input_shape[TL], num_epochs, train_loader, val_loader, test_loader1, test_loader3, test_loader5)
    
        #將gpu資料取回
        train_loss = torch.stack(train_loss,dim=-1).cpu()
        train_accuracy = torch.stack(train_accuracy,dim=-1).cpu()
        val_loss = torch.stack(val_loss,dim=-1).cpu()
        val_accuracy = torch.stack(val_accuracy,dim=-1).cpu()
    
        for i in range(num_epochs):
            # 寫入資料
            f.write("TL :{} fold: {} Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(TL+1 ,fold, i+1, num_epochs, train_loss[i], train_accuracy[i], val_loss[i], val_accuracy[i], test1_accuracy[i], test3_accuracy[i], test5_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        train_loss_TL.append(train_loss)
        train_accuracy_TL.append(train_accuracy)
        val_loss_TL.append(val_loss)
        val_accuracy_TL.append(val_accuracy)
        test1_accuracy_TL.append(test1_accuracy)
        test3_accuracy_TL.append(test3_accuracy)
        test5_accuracy_TL.append(test5_accuracy)
    modelName = "Fold_" + str(fold) + "_TF.pkl"
    torch.save(model, modelName)
    train_loss_array.append(train_loss_TL)
    train_accuracy_array.append(train_accuracy_TL)
    val_loss_array.append(val_loss_TL)
    val_accuracy_array.append(val_accuracy_TL)
    test1_accuracy_array.append(test1_accuracy_TL)
    test3_accuracy_array.append(test3_accuracy_TL)
    test5_accuracy_array.append(test5_accuracy_TL)
f.close()

----------------------------------------------------------------------------------------------------------------
TL:  3 fold:  1
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/30 Train_Loss: 1.010204792022705 Train_acc: 42.187500% Val_Loss: 1.071298599243164 Val_accuracy: 58.750000%
Test accuracy of the model: 0.000000%
Test accuracy of the model: 0.000000%
Test accuracy of the model: 0.000000%
Train Epoch: 2/30 Train_Loss: 0.7859859466552734 Train_acc: 74.062500% Val_Loss: 0.8766403794288635 Val_accuracy: 67.500000%
Test accuracy of the model: 0.000000%
Test accuracy of the model: 0.000000%
Test accuracy of the model: 0.000000%
Train Epoch: 3/30 Train_Loss: 0.4225386083126068 Train_acc: 83.125000% Val_Loss: 1.0468512773513794 Val_accuracy: 76.250000%
Test accuracy of the model: 60.600000%
Test accuracy of the model: 69.333333%
Test accuracy of the model: 70.000000%
Train Epoch: 4/30 Train_Loss: 0.35483047

Test accuracy of the model: 92.666667%
Test accuracy of the model: 97.000000%
Train Epoch: 3/30 Train_Loss: 0.15931323170661926 Train_acc: 95.625008% Val_Loss: 0.8971055150032043 Val_accuracy: 78.333336%
Test accuracy of the model: 91.600000%
Test accuracy of the model: 97.666667%
Test accuracy of the model: 99.000000%
Train Epoch: 4/30 Train_Loss: 0.15756279230117798 Train_acc: 95.729172% Val_Loss: 1.2476978302001953 Val_accuracy: 71.250000%
Test accuracy of the model: 91.000000%
Test accuracy of the model: 97.333333%
Test accuracy of the model: 99.000000%
Train Epoch: 5/30 Train_Loss: 0.21710704267024994 Train_acc: 94.270836% Val_Loss: 0.6686375141143799 Val_accuracy: 77.916672%
Test accuracy of the model: 89.200000%
Test accuracy of the model: 98.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 6/30 Train_Loss: 0.09761804342269897 Train_acc: 95.937508% Val_Loss: 1.2803902626037598 Val_accuracy: 68.750000%
Test accuracy of the model: 91.600000%
Test accuracy of the model: 

Test accuracy of the model: 94.800000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 6/30 Train_Loss: 0.1424606889486313 Train_acc: 92.937500% Val_Loss: 1.8365671634674072 Val_accuracy: 62.750000%
Test accuracy of the model: 93.200000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 7/30 Train_Loss: 0.16537277400493622 Train_acc: 91.937500% Val_Loss: 2.675199508666992 Val_accuracy: 68.000000%
Test accuracy of the model: 93.800000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 99.000000%
Train Epoch: 8/30 Train_Loss: 0.18044449388980865 Train_acc: 93.500000% Val_Loss: 2.1291935443878174 Val_accuracy: 64.750000%
Test accuracy of the model: 95.400000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 100.000000%
Train Epoch: 9/30 Train_Loss: 0.13938702642917633 Train_acc: 92.750000% Val_Loss: 2.4171955585479736 Val_accuracy: 64.250000%
Test accuracy of the model:

Test accuracy of the model: 70.200000%
Test accuracy of the model: 79.666667%
Test accuracy of the model: 88.000000%
Train Epoch: 9/30 Train_Loss: 0.13167139887809753 Train_acc: 95.625000% Val_Loss: 0.5500561594963074 Val_accuracy: 86.250000%
Test accuracy of the model: 77.400000%
Test accuracy of the model: 81.666667%
Test accuracy of the model: 86.000000%
Train Epoch: 10/30 Train_Loss: 0.17889375984668732 Train_acc: 97.500000% Val_Loss: 0.44849076867103577 Val_accuracy: 85.000000%
Test accuracy of the model: 93.600000%
Test accuracy of the model: 96.333333%
Test accuracy of the model: 99.000000%
Train Epoch: 11/30 Train_Loss: 0.06870195269584656 Train_acc: 99.062500% Val_Loss: 0.7633224129676819 Val_accuracy: 87.500000%
Test accuracy of the model: 95.000000%
Test accuracy of the model: 97.333333%
Test accuracy of the model: 99.000000%
Train Epoch: 12/30 Train_Loss: 0.083518847823143 Train_acc: 100.000000% Val_Loss: 0.6840187907218933 Val_accuracy: 86.250000%
Test accuracy of the mode

Train Epoch: 11/30 Train_Loss: 0.007849231362342834 Train_acc: 98.333336% Val_Loss: 0.6436554193496704 Val_accuracy: 88.333336%
Test accuracy of the model: 90.000000%
Test accuracy of the model: 98.333333%
Test accuracy of the model: 100.000000%
Train Epoch: 12/30 Train_Loss: 0.059567298740148544 Train_acc: 98.541672% Val_Loss: 0.24779462814331055 Val_accuracy: 90.833336%
Test accuracy of the model: 92.000000%
Test accuracy of the model: 99.333333%
Test accuracy of the model: 100.000000%
Train Epoch: 13/30 Train_Loss: 0.009508754126727581 Train_acc: 98.750008% Val_Loss: 0.2875555455684662 Val_accuracy: 86.666672%
Test accuracy of the model: 95.000000%
Test accuracy of the model: 100.000000%
Test accuracy of the model: 100.000000%
Train Epoch: 14/30 Train_Loss: 0.04405582323670387 Train_acc: 98.750008% Val_Loss: 0.14493601024150848 Val_accuracy: 88.750008%
Test accuracy of the model: 95.400000%
Test accuracy of the model: 100.000000%
Test accuracy of the model: 100.000000%
Train Epoch: 

...

Test accuracy of the model: 90.600000%
Test accuracy of the model: 97.666667%
Test accuracy of the model: 99.000000%
Train Epoch: 26/30 Train_Loss: 0.006442433688789606 Train_acc: 100.000000% Val_Loss: 0.10266811400651932 Val_accuracy: 91.250000%
Test accuracy of the model: 89.400000%
Test accuracy of the model: 97.333333%
Test accuracy of the model: 99.000000%
Train Epoch: 27/30 Train_Loss: 0.006389655638486147 Train_acc: 100.000000% Val_Loss: 0.018718432635068893 Val_accuracy: 92.500000%
Test accuracy of the model: 90.200000%
Test accuracy of the model: 98.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 28/30 Train_Loss: 0.00427249725908041 Train_acc: 100.000000% Val_Loss: 0.08495885878801346 Val_accuracy: 93.750000%
Test accuracy of the model: 89.800000%
Test accuracy of the model: 98.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 29/30 Train_Loss: 0.005932207684963942 Train_acc: 100.000000% Val_Loss: 0.10386249423027039 Val_accuracy: 91.250000%
Test accurac

Train Epoch: 28/30 Train_Loss: 0.04238292947411537 Train_acc: 98.645836% Val_Loss: 0.3340310752391815 Val_accuracy: 89.166672%
Test accuracy of the model: 96.000000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 29/30 Train_Loss: 0.03639359027147293 Train_acc: 98.645836% Val_Loss: 0.29327598214149475 Val_accuracy: 87.916672%
Test accuracy of the model: 96.000000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 30/30 Train_Loss: 0.03423108905553818 Train_acc: 98.645836% Val_Loss: 0.5557040572166443 Val_accuracy: 90.000008%
Test accuracy of the model: 96.000000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
----------------------------------------------------------------------------------------------------------------
TL:  1 fold:  4
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/30 Trai

Test accuracy of the model: 98.600000%
Test accuracy of the model: 100.000000%
Test accuracy of the model: 100.000000%
----------------------------------------------------------------------------------------------------------------
TL:  3 fold:  5
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/30 Train_Loss: 1.0694844722747803 Train_acc: 43.437500% Val_Loss: 1.3613004684448242 Val_accuracy: 65.000000%
Test accuracy of the model: 0.000000%
Test accuracy of the model: 0.000000%
Test accuracy of the model: 0.000000%
Train Epoch: 2/30 Train_Loss: 0.671675443649292 Train_acc: 72.187500% Val_Loss: 0.9234520196914673 Val_accuracy: 75.000000%
Test accuracy of the model: 0.000000%
Test accuracy of the model: 0.000000%
Test accuracy of the model: 0.000000%
Train Epoch: 3/30 Train_Loss: 0.5107513070106506 Train_acc: 82.500000% Val_Loss: 0.6893925070762634 Val_accuracy: 77.500000%
Test accuracy of the model: 0.000000%

Train Epoch: 2/30 Train_Loss: 0.223998561501503 Train_acc: 90.520836% Val_Loss: 0.44796445965766907 Val_accuracy: 73.333336%
Test accuracy of the model: 79.400000%
Test accuracy of the model: 94.666667%
Test accuracy of the model: 97.000000%
Train Epoch: 3/30 Train_Loss: 0.21365515887737274 Train_acc: 92.395836% Val_Loss: 0.6461698412895203 Val_accuracy: 72.916672%
Test accuracy of the model: 89.200000%
Test accuracy of the model: 96.666667%
Test accuracy of the model: 98.000000%
Train Epoch: 4/30 Train_Loss: 0.13503757119178772 Train_acc: 94.270836% Val_Loss: 0.6705992221832275 Val_accuracy: 81.666672%
Test accuracy of the model: 94.000000%
Test accuracy of the model: 98.000000%
Test accuracy of the model: 100.000000%
Train Epoch: 5/30 Train_Loss: 0.10312924534082413 Train_acc: 96.666672% Val_Loss: 0.6524539589881897 Val_accuracy: 82.500008%
Test accuracy of the model: 90.400000%
Test accuracy of the model: 98.000000%
Test accuracy of the model: 100.000000%
Train Epoch: 6/30 Train_Los

Test accuracy of the model: 94.200000%
Test accuracy of the model: 99.333333%
Test accuracy of the model: 99.000000%
Train Epoch: 5/30 Train_Loss: 0.2117818295955658 Train_acc: 91.000000% Val_Loss: 1.9421535730361938 Val_accuracy: 73.500000%
Test accuracy of the model: 95.800000%
Test accuracy of the model: 99.333333%
Test accuracy of the model: 100.000000%
Train Epoch: 6/30 Train_Loss: 0.18803851306438446 Train_acc: 91.375000% Val_Loss: 1.3954402208328247 Val_accuracy: 71.500000%
Test accuracy of the model: 94.200000%
Test accuracy of the model: 98.666667%
Test accuracy of the model: 99.000000%
Train Epoch: 7/30 Train_Loss: 0.19131246209144592 Train_acc: 91.187500% Val_Loss: 1.0644475221633911 Val_accuracy: 72.750000%
Test accuracy of the model: 93.200000%
Test accuracy of the model: 98.333333%
Test accuracy of the model: 100.000000%
Train Epoch: 8/30 Train_Loss: 0.27436530590057373 Train_acc: 92.250000% Val_Loss: 0.9809277653694153 Val_accuracy: 69.500000%
Test accuracy of the model:

In [4]:
#設定存檔位置
f = open("lastData.txt", "w")
for TL in range(3):
    train_loss_last = []
    train_accuracy_last = []
    val_loss_last = []
    val_accuracy_last = []
    test1_accuracy_last = []
    test3_accuracy_last = []
    test5_accuracy_last = []
    
    #整理出訓練完的數值
    for fold in range(5):
        train_loss_last.append(train_loss_array[fold][TL][num_epochs-1])
        train_accuracy_last.append(train_accuracy_array[fold][TL][num_epochs-1])
        val_loss_last.append(val_loss_array[fold][TL][num_epochs-1])
        val_accuracy_last.append(val_accuracy_array[fold][TL][num_epochs-1])
        test1_accuracy_last.append(test1_accuracy_array[fold][TL][num_epochs-1])
        test3_accuracy_last.append(test3_accuracy_array[fold][TL][num_epochs-1])
        test5_accuracy_last.append(test5_accuracy_array[fold][TL][num_epochs-1])
        # 寫入資料
        f.write("Fold: {} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(fold+1, train_loss_array[fold][TL][num_epochs-1], train_accuracy_array[fold][TL][num_epochs-1], val_loss_array[fold][TL][num_epochs-1], val_accuracy_array[fold][TL][num_epochs-1], test1_accuracy_array[fold][TL][num_epochs-1], test3_accuracy_array[fold][TL][num_epochs-1], test5_accuracy_array[fold][TL][num_epochs-1]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    # 寫入資料
    f.write("Max Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(max(train_loss_last), max(train_accuracy_last), max(val_loss_last), max(val_accuracy_last), max(test1_accuracy_last), max(test3_accuracy_last), max(test5_accuracy_last)))
    f.write("Min Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(min(train_loss_last), min(train_accuracy_last), min(val_loss_last), min(val_accuracy_last), min(test1_accuracy_last), min(test3_accuracy_last), min(test5_accuracy_last)))
    f.write("Avg Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(np.mean(train_loss_last), np.mean(train_accuracy_last), np.mean(val_loss_last), np.mean(val_accuracy_last), np.mean(test1_accuracy_last), np.mean(test3_accuracy_last), np.mean(test5_accuracy_last)))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
f.close()